<a href="https://colab.research.google.com/github/mrohith29/PhysiSolve/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step - 1
### spplitting the dataset into training, testing and evaluation

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import json
import random

RANDOM_SEED = 42
random.seed(RANDOM_SEED)

with open(r"/content/drive/MyDrive/dataset/high_school_physics.json", "r", encoding="utf-8") as f:
    data = json.load(f)

random.shuffle(data)

train_size = int(0.7 * len(data))
test_size = int(0.15 * len(data))
train_data = data[:train_size]
test_data = data[train_size:train_size + test_size]
eval_data = data[train_size + test_size:]

# Save train, test and eval sets
with open(r"/content/drive/MyDrive/dataset/train.json", "w", encoding="utf-8") as f:
    json.dump(train_data, f, indent=4)

with open(r"/content/drive/MyDrive/dataset/test.json", "w", encoding="utf-8") as f:
    json.dump(test_data, f, indent=4)

with open(r"/content/drive/MyDrive/dataset/eval.json", "w", encoding="utf-8") as f:
    json.dump(eval_data, f, indent=4)

import json
import random

RANDOM_SEED = 42
random.seed(RANDOM_SEED)

with open(r"/content/drive/MyDrive/dataset/high_school_physics.json", "r", encoding="utf-8") as f:
    data = json.load(f)

random.shuffle(data)

train_size = int(0.70 * len(data))
test_size = int(0.15 * len(data))
train_data = data[:train_size]
test_data = data[train_size:train_size + test_size]
eval_data = data[train_size + test_size:]

# Save train, test, and evaluation sets
with open(r"/content/drive/MyDrive/dataset/train.json", "w", encoding="utf-8") as f:
    json.dump(train_data, f, indent=4)

with open(r"/content/drive/MyDrive/dataset/test.json", "w", encoding="utf-8") as f:
    json.dump(test_data, f, indent=4)

with open(r"/content/drive/MyDrive/dataset/eval.json", "w", encoding="utf-8") as f:
    json.dump(eval_data, f, indent=4)

print(f"Dataset split into {len(train_data)} training, {len(test_data)} testing, and {len(eval_data)} evaluation samples.")

Dataset split into 280 training, 60 testing, and 60 evaluation samples.


In [3]:
print(test_data[0])
print(test_data[3])

{'id': 261, 'question': 'A 10 Ω resistor is connected across a 15 V battery. What is the current flowing through the resistor?', 'subject': 'Electrostatics and Current Electricity', 'choices': ['0.5 A', '1 A', '1.5 A', '2 A'], 'answer': 'C', 'explanation': 'I = V/R = 15/10 = 1.5 A', 'dataset': 'high_school_physics'}
{'id': 338, 'question': 'A force of 80 N is applied to a 8 kg block. What is the acceleration of the block?', 'subject': 'Mechanics', 'choices': ['5 m/s²', '10 m/s²', '15 m/s²', '20 m/s²'], 'answer': 'B', 'explanation': 'F = ma => a = F/m = 80/8 = 10 m/s²', 'dataset': 'high_school_physics'}


### Few-Shot Evaluation of Flan-T5 from Hugging Face

In [4]:
from transformers import pipeline

qa_pipeline = pipeline("text2text-generation", model="google/flan-t5-large")

def evaluate_model(model, dataset):
    correct = 0
    total = len(dataset)

    for item in dataset:
        question = item["question"]
        idx = ord(item["answer"])-ord("A")
        correct_answer = item["choices"][idx]

        prompt = f"Give me the final answer without any explaination, just the couple of words with units that directly show the answer for the Question: {question} with Choices: {', '.join(item['choices'])} Answer:"
        prediction = model(prompt, max_length=20, truncation=True)[0]["generated_text"]

        # print(correct_answer, prediction, item["id"])
        if correct_answer in prediction:
          correct += 1

    accuracy = (correct / total) * 100
    return accuracy

zero_shot_accuracy = evaluate_model(qa_pipeline, test_data)
print(f"Zero-Shot Accuracy: {zero_shot_accuracy:.2f}%")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


Zero-Shot Accuracy: 33.33%


### Fine-Tuning Flan-T5 using LoRA

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
from torch.utils.data import Dataset
import json
import os
import time

# Disable W&B logging
os.environ["WANDB_DISABLED"] = "true"

# Load model and tokenizer
model_name = "google/flan-t5-large"
model = T5ForConditionalGeneration.from_pretrained(model_name)
model.config.use_cache = False  # Fix past_key_values deprecation warning
tokenizer = T5Tokenizer.from_pretrained(model_name, legacy=False)

# Create custom dataset
class PhysicsDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        prompt = f"Question: {item['question']} Choices: {', '.join(item['choices'])} Answer:"
        idx = ord(item["answer"][0])-ord("A")
        target = item["choices"][idx]

        encodings = self.tokenizer(prompt, truncation=True, padding="max_length", max_length=self.max_length, return_tensors="pt")
        target_encodings = self.tokenizer(target, truncation=True, padding="max_length", max_length=self.max_length, return_tensors="pt")

        return {
            "input_ids": encodings["input_ids"].squeeze(),
            "attention_mask": encodings["attention_mask"].squeeze(),
            "labels": target_encodings["input_ids"].squeeze(),
        }

# Load json files
train_file_path = "/content/drive/MyDrive/dataset/train.json"
with open(train_file_path, "r", encoding="utf-8") as f:
    train_data = json.load(f)

# Prepare datasets
train_dataset = PhysicsDataset(train_data, tokenizer)

# Load evaluation data
eval_file_path = "/content/drive/MyDrive/dataset/eval.json"
with open(eval_file_path, "r", encoding="utf-8") as f:
    eval_data = json.load(f)

eval_dataset = PhysicsDataset(eval_data, tokenizer)

# Define output directories
model_save_dir = "/content/drive/MyDrive/dataset/trained_model"
results_dir = "/content/drive/MyDrive/dataset/results"

# Create directories if they don't exist
os.makedirs(model_save_dir, exist_ok=True)
os.makedirs(results_dir, exist_ok=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir=results_dir,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    save_steps=500,
    evaluation_strategy="epoch",
    logging_dir="./logs",
    run_name=f"flan-t5-{time.strftime('%Y%m%d-%H%M%S')}",
    report_to="none",
)

# Train model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

trainer.train()

# Save the trained model and tokenizer properly
model.save_pretrained(model_save_dir)
tokenizer.save_pretrained(model_save_dir)
print(f"Model and tokenizer saved to {model_save_dir}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


### Testing the trained model on test.json

In [6]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import json
import torch
import os
from tqdm import tqdm

# Paths
model_save_dir = "/content/drive/MyDrive/dataset/trained_model"
test_file_path = "/content/drive/MyDrive/dataset/test.json"  # Adjust if your test file has a different name

# First, let's check if the model files exist
print("Checking model directory contents:")
for root, dirs, files in os.walk(model_save_dir):
    for file in files:
        print(os.path.join(root, file))

# Load the model and tokenizer
try:
    model = T5ForConditionalGeneration.from_pretrained(model_save_dir)
    tokenizer = T5Tokenizer.from_pretrained(model_save_dir)
    print("Model and tokenizer loaded successfully!")
except Exception as e:
    print(f"Error loading model: {e}")
    # If model loading fails, load the original pretrained model
    print("Loading the base model instead...")
    model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large")
    tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large", legacy=False)

# Load test data
with open(test_file_path, "r", encoding="utf-8") as f:
    test_data = json.load(f)

print(f"Loaded {len(test_data)} test examples")

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Testing function
def evaluate_model(model, tokenizer, test_data, device, correct, total):

    for item in tqdm(test_data):
        prompt = f"Question: {item['question']} Choices: {', '.join(item['choices'])} Answer:"
        correct_idx = ord(item["answer"][0]) - ord("A")
        correct_answer = item["choices"][correct_idx]

        # Tokenize input
        input_ids = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=128).input_ids.to(device)

        # Generate output
        with torch.no_grad():
            outputs = model.generate(
                input_ids=input_ids,
                max_length=128,
                num_beams=4,
                early_stopping=True
            )

        # Decode output
        predicted_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Simple string match for evaluation
        if predicted_text.strip() == correct_answer.strip():
            correct += 1
        else:
            # Print some examples of wrong predictions for debugging
            if total < 5:  # Limit to just a few examples
                print(f"\nQuestion: {item['question']}")
                print(f"Choices: {', '.join(item['choices'])}")
                print(f"Correct Answer: {correct_answer}")
                print(f"Predicted: {predicted_text}")

        total += 1

    accuracy = correct / total if total > 0 else 0
    return accuracy

# Evaluate model
print("Evaluating model on test set...")
correct = 0
total = 0
accuracy = evaluate_model(model, tokenizer, test_data, device, correct, total)
print(f"Test Accuracy: {accuracy:.4f} ({correct}/{total})")

Checking model directory contents:
Error loading model: Error no file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt.index or flax_model.msgpack found in directory /content/drive/MyDrive/dataset/trained_model.
Loading the base model instead...
Loaded 60 test examples
Evaluating model on test set...


  2%|▏         | 1/60 [00:05<05:38,  5.74s/it]


Question: A 10 Ω resistor is connected across a 15 V battery. What is the current flowing through the resistor?
Choices: 0.5 A, 1 A, 1.5 A, 2 A
Correct Answer: 1.5 A
Predicted: 2 A


  3%|▎         | 2/60 [00:11<05:44,  5.94s/it]


Question: A force of 100 N is applied to a 50 kg object. What is the acceleration of the object?
Choices: 1 m/s², 2 m/s², 3 m/s², 4 m/s²
Correct Answer: 2 m/s²
Predicted: 4 m/s2


...


Question: A force of 80 N is applied to a 8 kg block. What is the acceleration of the block?
Choices: 5 m/s², 10 m/s², 15 m/s², 20 m/s²
Correct Answer: 10 m/s²
Predicted: 20 m/s2


  8%|▊         | 5/60 [00:21<03:35,  3.92s/it]


Question: Two resistors of 15Ω and 5Ω are connected in parallel. What is the equivalent resistance?
Choices: 1Ω, 2Ω, 3.75Ω, 5Ω
Correct Answer: 3.75Ω
Predicted: 5


100%|██████████| 60/60 [03:16<00:00,  3.27s/it]

Test Accuracy: 0.2667 (0/0)


In [2]:
import torch
import json
from transformers import T5ForConditionalGeneration, T5Tokenizer
from torch.utils.data import Dataset
import pickle

# Load trained model
model_path = "/content/drive/MyDrive/dataset/trained_model.pkl"
with open(model_path, "rb") as f:
    model = pickle.load(f)

# Load tokenizer
model_name = "google/flan-t5-large"
tokenizer = T5Tokenizer.from_pretrained(model_name, legacy=False)

# Load test dataset
test_file_path = "/content/drive/MyDrive/dataset/test.json"
with open(test_file_path, "r", encoding="utf-8") as f:
    test_data = json.load(f)

# Define test dataset class
class PhysicsDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        prompt = f"Question: {item['question']} Choices: {', '.join(item['choices'])} Answer:"
        idx = ord(item["answer"][0])-ord("A")  # Convert letter to index
        target = item["choices"][idx]

        encodings = self.tokenizer(prompt, truncation=True, padding="max_length", max_length=self.max_length, return_tensors="pt")
        target_encodings = self.tokenizer(target, truncation=True, padding="max_length", max_length=self.max_length, return_tensors="pt")

        return {
            "input_ids": encodings["input_ids"].squeeze(),
            "attention_mask": encodings["attention_mask"].squeeze(),
            "labels": target_encodings["input_ids"].squeeze(),
            "correct_answer": target
        }

# Prepare test dataset
test_dataset = PhysicsDataset(test_data, tokenizer)

# Function to evaluate accuracy
def evaluate_accuracy(model, dataset, tokenizer):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0

    for item in dataset:
        input_ids = item["input_ids"].unsqueeze(0).to("cuda" if torch.cuda.is_available() else "cpu")
        attention_mask = item["attention_mask"].unsqueeze(0).to("cuda" if torch.cuda.is_available() else "cpu")

        # Generate prediction
        with torch.no_grad():
            output = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=128)

        # Decode predicted and actual answer
        predicted_answer = tokenizer.decode(output[0], skip_special_tokens=True)
        correct_answer = item["correct_answer"]

        # Compare answers
        if predicted_answer.strip().lower() == correct_answer.strip().lower():
            correct += 1
        total += 1

    accuracy = (correct / total) * 100
    return accuracy

# Run evaluation
accuracy = evaluate_accuracy(model, test_dataset, tokenizer)
print(f"Test Accuracy: {accuracy:.2f}%")


EOFError: Ran out of input